# Speech Emotion Recognition Demo

A demo containing all steps of speech emotion recoginition using the EmoDB dataset. The demo has been divided into 3 phases:
- Phase 1: Loading audio files and extracting metadata
- Phase 2: Embedding Extraction
- Phase 3: Downstream Task - Speech Emotion Recognotion



### About EmoDB:
- A German database of emotional speech
- 800 recordings
- 10 actors (5 males and 5 females)
- 7 emotions: anger, neutral, fear, boredom, happiness, sadness, disgust

### References:
- Dataset: http://emodb.bilderbar.info/index-1280.html
- Paper: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.130.8506&rep=rep1&type=pdf

Lets start by importing a few packages.


### Importing packages

In [ ]:
!pip install -q speechbrain
!pip install -q  transformers
!git clone -q https://github.com/GasserElbanna/serab-byols.git
!python3 -m pip install -q -e ./serab-byols

!pip install -q tqdm==4.60.0
!pip install -q opensmile

     |████████████████████████████████| 496 kB 4.9 MB/s 
     |████████████████████████████████| 1.3 MB 57.5 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 750.6 MB 10 kB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
     |████████████████████████████████| 109 kB 57.6 MB/s 
     |████████████████████████████████| 546 kB 39.3 MB/s 
     |████████████████████████████████| 3.7 MB 49.1 MB/s 
     |████████████████████████████████| 3.7 MB 46.6 MB/s 
     |████████████████████████████████| 2.9 MB 38.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
  

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from glob import glob
from random import sample

import librosa
import soundfile as sf

import torch
import opensmile
import serab_byols
from transformers import Wav2Vec2Model, HubertModel

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')


In [ ]:
! pip install -q kaggle

from google.colab import files
files.upload()

# Name directory
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving utilities.py to utilities.py
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


# Phase 1: Loading audio files and extracting metadata

Includes downloading the dataset, loading audio files, resampling audio files, extracting metadata



### Defining a function for loading and resampling audio files

In [ ]:
# Defining a function for loading and resampling audio files

def load_audio_files(audio_files, resampling_frequency=16000, audio_list=None):
  '''
  Loads and resamples audio files 
  
  Parameters
  ------------
  audio_files: string
      The paths of the wav files 
  resampling_frequency: integer
      The frequency which all audios will be resampled to
  audio_list: list 
      The list of torch tensors of audios to which more audios need too be added, empty by default

  Returns
  ------------
  audio_list: list
      A list of torch tensors, one array for each audio file

  '''
  # Making audio_list
  if audio_list is None:
    audio_list = []

  # Resampling
  for audio in audio_files:
    signal, fs = librosa.load(audio, sr=resampling_frequency)
    audio_list.append(torch.from_numpy(signal))
      
  return audio_list
        

### Loading and resampling audiofiles and collecting metadata on EmoDB dataset

In [ ]:
# Phase_1
# Load dataset
! kaggle datasets download -q -d piyushagni5/berlin-database-of-emotional-speech-emodb
! unzip -q berlin-database-of-emotional-speech-emodb.zip

# Resample dataset
audio_files = glob('/content/wav/*.wav')
audio_list = load_audio_files(audio_files, resampling_frequency=16000)

# Extracting metadata (speakers and labels)
labels = np.array(list(map(lambda x: os.path.basename(x).split('.')[0][-2], audio_files)))
speakers = np.array(list(map(lambda x: os.path.basename(x)[:2], audio_files)))

# ---------------------------------------------------------------------------------------------------

# Verify phase_1
print('Number of audio files: {}'.format(len(audio_list)))
print('Number of speaker classes: {}'.format(len(set(speakers))))
print('Speaker classes: {}'.format(set(speakers)))
print('Number of speakers: {}'.format(len(speakers)))
print('Speakers:')
print(speakers)
print('Number of label classes: {}'.format(len(set(labels))))
print('Label classes: {}'.format(set(labels)))
print('Number of labels: {}'.format(len(labels)))
print('Labels:')
print(labels)

Number of audio files: 535
Number of speaker classes: 10
Speaker classes: {'15', '03', '14', '12', '16', '09', '11', '13', '10', '08'}
Number of speakers: 535
Speakers:
['03' '14' '09' '15' '13' '13' '09' '16' '16' '13' '12' '08' '03' '15'
 '16' '08' '13' '15' '14' '12' '09' '11' '14' '09' '14' '11' '15' '11'
 '13' '10' '13' '10' '15' '14' '12' '15' '11' '14' '08' '15' '12' '08'
 '11' '13' '14' '03' '16' '13' '16' '16' '09' '16' '12' '14' '12' '15'
 '08' '14' '03' '08' '14' '09' '14' '12' '13' '13' '10' '03' '16' '14'
 '14' '08' '11' '10' '08' '14' '15' '15' '10' '09' '09' '03' '13' '14'
 '03' '03' '10' '16' '10' '16' '03' '11' '11' '08' '10' '11' '14' '08'
 '16' '10' '14' '03' '12' '14' '08' '10' '11' '11' '03' '14' '11' '16'
 '10' '13' '15' '13' '11' '12' '11' '09' '16' '14' '16' '16' '11' '14'
 '11' '16' '03' '14' '15' '15' '14' '03' '13' '09' '12' '03' '11' '12'
 '13' '10' '10' '08' '14' '12' '08' '09' '08' '09' '13' '09' '03' '11'
 '16' '03' '09' '13' '12' '11' '10' '09' '11' '08'

# Phase 2: Embedding Extraction
Includes extracting features using
- Deep learning based methods: Hybrid BYOL-S
- DSP based methods: openSMILE compare, openSMILE egemaps

### Audio embeddings extraction functions

In [ ]:
# Defining a function for generating audio embedding extraction models

def audio_embeddings_model(model_name):
  '''
  Generates model for embedding extraction 
  
  Parameters
  ------------
  mode_name: string
      The model to used, could be 'hybrid_byols', 'compare' or 'egemaps'

  Returns
  ------------
  model: object
      The embedding extraction model
  '''
  if model_name=='hybrid_byols':
    model_name = 'cvt'
    checkpoint_path = "serab-byols/checkpoints/cvt_s1-d1-e64_s2-d1-e256_s3-d1-e512_BYOLAs64x96-osandbyolaloss6373-e100-bs256-lr0003-rs42.pth"
    model = serab_byols.load_model(checkpoint_path, model_name)
  elif model_name=='compare':
    model = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.Functionals,
    )
  elif model_name=='egemaps':
    model = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals,
    )
  return model


# Defining a function for embedding exctraction from the audio list

def audio_embeddings(audio_list, model_name, model, sampling_rate=16000):
  '''
  Loads and resamples audio files 
  
  Parameters
  ------------
  audio_list: list
      A list of arrays, one array for each audio file
  model_name: string
      The model to used, could be 'hybrid_byols', 'compare' or 'egemaps'
  model: object
      The embedding extraction model generated by audio_embeddings_model function
  sampling_rate: int
      The sampling rate, 16 kHz by default

  Returns
  ------------
  embeddings_array: array
      The array containg embeddings of all audio_files, dimension (number of audio files × n_feats)
      
  '''
  if model_name=='hybrid_byols':
    embeddings_array = serab_byols.get_scene_embeddings(audio_list, model)
  else:
    embeddings_list = []
    for i in tqdm(range(len(audio_list))):
      embeddings = model.process_signal(audio_list[i], sampling_rate)
      embeddings_list.append(torch.tensor(embeddings.values[0], dtype=torch.float32))
    embeddings_array = torch.stack(embeddings_list)
  return embeddings_array


### Audio embeddings extraction on EmoDB

In [ ]:
# Phase_2

# Hybrid BYOLS
model = audio_embeddings_model(model_name='hybrid_byols')
embeddings_array_byols = audio_embeddings(audio_list, model_name='hybrid_byols', model=model)

# EmoDB compare
model = audio_embeddings_model(model_name='compare')
embeddings_array_compare = audio_embeddings(audio_list, model_name='compare', model=model)

# EmoDB egemaps
model = audio_embeddings_model(model_name='egemaps')
embeddings_array_egemaps = audio_embeddings(audio_list, model_name='egemaps', model=model)

# ---------------------------------------------------------------------------------------------------

# Verify Phase_2
models = ['hybrid_byols', 'compare', 'egemaps']
embeddings_arrays = {'hybrid_byols': embeddings_array_byols, 'compare':embeddings_array_compare, 'egemaps':embeddings_array_egemaps}

for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of the embeddings array is {}'.format(embeddings_arrays[model].shape))
  print('The embeddings array is: ')
  print((embeddings_arrays[model]))


100%|██████████| 535/535 [01:02<00:00,  8.62it/s]



MODEL: hybrid_byols

The shape of the embeddings array is torch.Size([535, 2048])
The embeddings array is: 
tensor([[ 3.3003,  5.1818,  0.9551,  ...,  5.0923, -1.7271,  4.4259],
        [ 3.7851,  4.7061,  1.2274,  ...,  4.3181, -0.3392,  3.7923],
        [ 4.6339,  4.9679,  1.4998,  ...,  4.3758,  0.0359,  3.8127],
        ...,
        [ 3.4200,  4.8069,  0.5398,  ...,  4.8588,  0.4056,  4.0666],
        [ 5.8241,  5.3683,  1.2110,  ...,  5.1313,  1.4869,  3.6378],
        [ 6.2243,  4.5836,  1.7008,  ...,  4.7435, -0.9581,  3.1392]])


MODEL: compare

The shape of the embeddings array is torch.Size([535, 6373])
The embeddings array is: 
tensor([[2.7397e+00, 1.9208e-01, 4.5941e-01,  ..., 5.4328e+01, 1.0450e+02,
         5.5951e+01],
        [2.6716e+00, 2.0376e-01, 5.9561e-01,  ..., 4.5818e+01, 1.0355e+02,
         4.9328e+01],
        [3.6595e+00, 6.7089e-01, 1.2658e-02,  ..., 6.3352e+01, 1.4530e+02,
         6.7398e+01],
        ...,
        [2.9835e+00, 6.8056e-01, 2.2917e-01,  .

# Phase 3: Downstream Task - Speech Emotion Recognotion
Includes speaker normalisation, train test splitting and hyperparameter tuning using logistic regression, SVM and random forest classification 

### Speaker normalisation functions

In [ ]:
# Defining a function for speaker normalisation using standard scaler

def speaker_normalisation(embeddings_array, speakers):
  '''
  Normalises embeddings_array for each speaker
  
  Parameters
  ------------
  embeddings_array: array
      The array of embeddings, one row for each audio file
  speakers: list 
      The list of speakers

  Returns
  ------------
  embeddings_array: array
      The array containg normalised embeddings of all audio_files, dimension (number of audio files × n_feats)
      
  '''
  speaker_ids = set(speakers)
  for speaker_id in speaker_ids:
    speaker_embeddings_indices = np.where(np.array(speakers)==speaker_id)[0]
    speaker_embeddings = embeddings_array[speaker_embeddings_indices,:]
    scaler = StandardScaler()
    normalised_speaker_embeddings = scaler.fit_transform(speaker_embeddings)
    embeddings_array[speaker_embeddings_indices] = torch.tensor(normalised_speaker_embeddings).float()
  return embeddings_array


### Speaker normalisation on EmoDB

In [ ]:
# Normalised arrays
normalised_embeddings_byols = speaker_normalisation(embeddings_array_byols, speakers)
normalised_embeddings_compare= speaker_normalisation(embeddings_array_compare, speakers)
normalised_embeddings_egemaps = speaker_normalisation(embeddings_array_egemaps, speakers)


# Verifying normalised_embeddings_arrays

normalised_embeddings_arrays = {'hybrid_byols': normalised_embeddings_byols, 'compare':normalised_embeddings_compare, 'egemaps':normalised_embeddings_egemaps}

for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of the normalised embeddings array is: {}'.format(normalised_embeddings_arrays[model].shape))
  print('Normalised Embeddings Array:')
  print((normalised_embeddings_arrays[model]))
  columnwise_mean = torch.mean(normalised_embeddings_arrays[model], 0)
  print('Columnwise_mean:')
  print(columnwise_mean)
  if torch.all(columnwise_mean < 10**(-6)):
    print('PASSED: All means are less than 10**-6')
  else:
    print('FAILED: All means are NOT less than 10**-6')




MODEL: hybrid_byols

The shape of the normalised embeddings array is: torch.Size([535, 2048])
Normalised Embeddings Array:
tensor([[-1.4416,  1.1102,  0.5387,  ...,  0.0447, -1.5613,  1.9950],
        [-1.2288,  0.2675, -0.2237,  ..., -0.2757,  0.2524,  0.9853],
        [-0.3926,  0.0199,  0.1643,  ..., -0.7147,  0.0252,  0.5424],
        ...,
        [-1.6562, -0.2163, -1.9813,  ..., -0.1907,  0.7741,  0.9494],
        [ 0.2894,  0.1747, -0.6020,  ...,  0.2717,  1.8460, -0.1170],
        [ 1.2788, -0.2910,  0.5713,  ..., -0.1577, -0.8340, -0.7457]])
Columnwise_mean:
tensor([-3.5651e-09,  8.9128e-10,  0.0000e+00,  ..., -2.6739e-09,
        -2.8967e-09,  3.5651e-09])
PASSED: All means are less than 10**-6


MODEL: compare

The shape of the normalised embeddings array is: torch.Size([535, 6373])
Normalised Embeddings Array:
tensor([[-0.8150, -0.7810,  0.5549,  ..., -0.8784, -0.8868, -0.6129],
        [-0.7592, -0.5905,  0.4798,  ..., -1.7397, -1.2616, -0.9612],
        [ 0.7708,  0.737

### Train Test splitting functions

In [ ]:
# Defining a function for splitting into train set and test set with diferent speakers in each set

def split_train_test(normalised_embeddings_array, labels, speakers, test_size = 0.30):
  '''
  Splits into training and testing set with different speakers

  Parameters
  ------------
  normalised_embeddings_array: torch tensor
    The tensor containing normalised embeddings 
  labels: list of strings
    The list of emotions corresponding to audio files
  speakers: list 
    The list of speakers
  test_size: float 
    The fraction of embeddings and labels to put in the test set

  Returns
  ------------
  X_train: torch tensor
    The normalised embeddings that will be used for training
  X_test: torch tensor
    The normalised embeddings that will be used for testing
  y_train: list
   The labels that will be used for training
  y_test: list
   The labels that will be used for testing

  '''
  # unique speakers in this dataset
  all_speakers = set(speakers)
  # unique speakers in test set
  test_speakers = sample(all_speakers, int(test_size*len(all_speakers)))

  test_speakers_indices = []
  train_speakers_indices = []

  for speaker in all_speakers:
      if speaker in test_speakers:
          speaker_indices = np.where(np.array(speakers)==speaker)[0]
          test_speakers_indices.extend(speaker_indices)
      else:
          speaker_indices = np.where(np.array(speakers)==speaker)[0]
          train_speakers_indices.extend(speaker_indices)

  X_train = normalised_embeddings_array[train_speakers_indices]
  X_test = normalised_embeddings_array[test_speakers_indices]

  y_train = [0 for i in range(len(train_speakers_indices))]
  y_test = [0 for i in range(len(test_speakers_indices))]

  for i,index in enumerate(train_speakers_indices):
      y_train[i] = labels[index]
  for i,index in enumerate(test_speakers_indices):
      y_test[i] = labels[index]

  return X_train, X_test, y_train, y_test


### Train Test splitting on EmoDB

In [ ]:
# Phase_3: Train Test splitting

X_train_byols, X_test_byols, y_train_byols, y_test_byols = split_train_test(normalised_embeddings_byols, labels, speakers, test_size = 0.30)
X_train_compare, X_test_compare, y_train_compare, y_test_compare = split_train_test(normalised_embeddings_compare, labels, speakers, test_size = 0.30)
X_train_egemaps, X_test_egemaps, y_train_egemaps, y_test_egemaps = split_train_test(normalised_embeddings_egemaps, labels, speakers, test_size = 0.30)

X_trains = {'hybrid_byols':X_train_byols, 'compare':X_train_compare, 'egemaps':X_train_egemaps}
X_tests = {'hybrid_byols':X_test_byols, 'compare':X_test_compare, 'egemaps':X_test_egemaps}
y_trains = {'hybrid_byols':y_train_byols, 'compare':y_train_compare, 'egemaps':y_train_egemaps}
y_tests = {'hybrid_byols':y_test_byols, 'compare':y_test_compare, 'egemaps':y_test_egemaps}

# Verify
for model in models:
  print()
  print()
  print('MODEL: {}'.format(model))
  print()
  print('The shape of X_train is: {}'.format(X_trains[model].shape))
  print('X_train:')
  print(X_trains[model])
  print()
  print('The shape of X_test is: {}'.format(X_tests[model].shape))
  print('X_test:')
  print(X_tests[model])
  print()
  print('The length of y_train is: {}'.format(len(y_trains[model])))
  print('y_train:')
  print(y_trains[model])
  print()
  print('The length of y_test is: {}'.format(len(y_tests[model])))
  print('y_test:')
  print(y_tests[model])




MODEL: hybrid_byols

The shape of X_train is: torch.Size([368, 2048])
X_train:
tensor([[-1.6433,  0.4363, -0.5681,  ...,  0.1022, -0.6459,  0.0323],
        [ 1.5184, -0.4476,  0.6933,  ...,  0.0404, -0.0185,  0.5765],
        [-0.9973, -1.9792, -2.0397,  ..., -0.7453, -0.4625,  0.5128],
        ...,
        [ 0.2091,  0.2490, -2.3426,  ...,  1.7335,  1.7397, -1.2892],
        [-0.6214,  0.0556,  0.4667,  ...,  0.0756, -1.3667,  0.3865],
        [-0.0228,  0.4206, -1.4976,  ...,  0.7505,  0.6567,  1.0557]])

The shape of X_test is: torch.Size([167, 2048])
X_test:
tensor([[ 0.6146,  3.7346, -0.2716,  ...,  0.4125, -1.6326,  0.9995],
        [-0.2403, -1.2074,  0.7718,  ...,  0.7722,  1.0866, -1.1885],
        [-0.0483,  0.6215,  0.0361,  ...,  0.0805, -0.4190,  0.2383],
        ...,
        [ 0.1686,  1.0368, -0.5102,  ...,  0.3500,  1.8225, -0.9022],
        [-1.6562, -0.2163, -1.9813,  ..., -0.1907,  0.7741,  0.9494],
        [ 0.2894,  0.1747, -0.6020,  ...,  0.2717,  1.8460, -0.11

### Hyperparameter tuning functions

In [ ]:
# Defining a function for hyperparameter tuning and getting the accuracy on the test set

def get_hyperparams(X_train, X_test, y_train, y_test, classifier, parameters):
  '''
  Splits into training and testing set with different speakers

  Parameters
  ------------
  X_train: torch tensor
    The normalised embeddings that will be used for training
  X_test: torch tensor
    The normalised embeddings that will be used for testing
  y_train: list
    The labels that will be used for training
  y_test: list
    The labels that will be used for testing
  classifier: object
    The instance of the classification model 
  parameters: dictionary
    The dictionary of parameters for GridSearchCV 

  Returns
  ------------
    The dictionary of the best hyperparameters
  
  '''
  grid = GridSearchCV(classifier, param_grid = parameters, cv=5, scoring='recall_macro')                     
  grid.fit(X_train,y_train)
  print('recall_macro :',grid.best_score_)
  print('Best Parameters: {}'.format(grid.best_params_))
  print('recall_macro on test_set: {}'.format(grid.score(X_test, y_test)))
  predictions = grid.predict(X_test)
  print(classification_report(y_test, predictions))
  return grid.score(X_test, y_test)


### Hyperparameter tuning and getting recall_macro on EmoDB

In [ ]:
results = {'Logistic Regression': {'hybrid_byols': 0, 'compare': 0, 'egemaps': 0},
        'Support Vector Machine': {'hybrid_byols': 0, 'compare': 0, 'egemaps': 0},
        'Random Forest Classification': {'hybrid_byols': 0, 'compare': 0, 'egemaps': 0}}
        

In [ ]:
# Logistic Regression
print('Logistic Regression:')
classifier = LogisticRegression()
parameters = {'penalty' : ['l1','l2'], 'C': np.logspace(-4,2,7), 'solver': ['newton-cg', 'lbfgs', 'liblinear']}
for model in models:
  print('MODEL: {}'.format(model))
  results['Logistic Regression'][model] = np.round(100*get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters),1)
print()

# Support Vector Machine
print('Support Vector Machine:')
classifier = SVC()
parameters = {'C': np.logspace(-2,3,6), 'gamma': np.logspace(-5,2,8), 'kernel':['rbf','poly','sigmoid','linear']}
for model in models:
  print('MODEL: {}'.format(model))
  results['Support Vector Machine'][model] = np.round(100*get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters),1)
print()

# Random Forest Classifier
print('Random Forest Classifier:')
classifier = RandomForestClassifier()
parameters = {'n_estimators' : [50,100,200], 'max_features' : ['auto', 'log2', 'sqrt'], 'bootstrap' : [True, False]}
for model in models:
  print('MODEL: {}'.format(model))
  results['Random Forest Classification'][model] = np.round(100*get_hyperparams(X_trains[model], X_tests[model], y_trains[model], y_tests[model], classifier, parameters),1)
print()


Logistic Regression:
MODEL: hybrid_byols
recall_macro : 0.8970017564135213
Best Parameters: {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
recall_macro on test_set: 0.873632866703061
              precision    recall  f1-score   support

           A       0.94      0.75      0.83        20
           E       1.00      0.81      0.89        21
           F       0.67      0.70      0.68        23
           L       1.00      0.97      0.98        29
           N       0.82      1.00      0.90        18
           T       0.90      1.00      0.95        18
           W       0.85      0.89      0.87        38

    accuracy                           0.87       167
   macro avg       0.88      0.87      0.87       167
weighted avg       0.88      0.87      0.87       167

MODEL: compare
recall_macro : 0.8199192404234422
Best Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
recall_macro on test_set: 0.8550031328320802
              precision    recall  f1-score   support

 

# Results

Summarising the results obtained by using Deep Learning based features (hybrid BYOL-S) and DSP based features (openSMILE comPare and openSMILE egemaps) on logistic regression, SVM, random forest classification

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df

,Logistic Regression,Support Vector Machine,Random Forest Classification
hybrid_byols,87.4,87.9,75.8
compare,85.5,84.7,76.2
egemaps,74.0,75.2,71.3
